# Keyboard System

> High-level API for rendering complete keyboard navigation systems.

In [ ]:
#| default_exp components.system

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
from fasthtml.common import Script, Div

from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager
from cjm_fasthtml_keyboard_navigation.js.generators import generate_keyboard_script
from cjm_fasthtml_keyboard_navigation.htmx.inputs import render_hidden_inputs
from cjm_fasthtml_keyboard_navigation.htmx.buttons import render_action_buttons
from cjm_fasthtml_keyboard_navigation.components.hints import render_keyboard_hints

## KeyboardSystem Result

In [ ]:
#| export
@dataclass
class KeyboardSystem:
    """Container for all keyboard navigation components."""
    script: Script                    # the keyboard navigation JavaScript
    hidden_inputs: Div                # hidden inputs for HTMX
    action_buttons: Div               # hidden action buttons for HTMX
    hints: Optional[Div] = None       # optional keyboard hints UI

    def all_components(self) -> tuple:  # all components as tuple
        """Return all components for easy unpacking into render."""
        components = [self.script, self.hidden_inputs, self.action_buttons]
        if self.hints:
            components.append(self.hints)
        return tuple(components)

## Render Keyboard System

In [ ]:
#| export
def render_keyboard_system(
    manager: ZoneManager,                        # the zone manager configuration
    url_map: dict[str, str],                     # action button ID -> URL
    target_map: dict[str, str],                  # action button ID -> target selector
    include_map: dict[str, str] | None = None,   # action button ID -> include selector
    swap_map: dict[str, str] | None = None,      # action button ID -> swap value
    show_hints: bool = True,                     # render keyboard hints UI
    hints_badge_style: str = "ghost",            # badge style for hints (ghost, outline, soft, dash)
    include_state_inputs: bool = False           # include state tracking inputs
) -> KeyboardSystem:                             # complete keyboard system
    """Render complete keyboard navigation system."""
    # Generate JavaScript
    script = Script(generate_keyboard_script(manager))
    
    # Generate hidden inputs
    hidden_inputs = render_hidden_inputs(
        manager,
        include_state=include_state_inputs
    )
    
    # Generate action buttons
    action_buttons = render_action_buttons(
        manager,
        url_map=url_map,
        target_map=target_map,
        include_map=include_map,
        swap_map=swap_map
    )
    
    # Generate hints
    hints = None
    if show_hints:
        hints = render_keyboard_hints(
            manager,
            badge_style=hints_badge_style
        )
    
    return KeyboardSystem(
        script=script,
        hidden_inputs=hidden_inputs,
        action_buttons=action_buttons,
        hints=hints
    )

In [ ]:
# Test render_keyboard_system
from fasthtml.common import to_xml
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction

browser = FocusZone(
    id="browser",
    item_selector="tr.item",
    data_attributes=("job-id",)
)
queue = FocusZone(
    id="queue",
    item_selector="li.item"
)

manager = ZoneManager(
    zones=(browser, queue),
    actions=(
        KeyAction(key=" ", htmx_trigger="toggle-btn", description="Select"),
        KeyAction(key="Delete", htmx_trigger="delete-btn", description="Remove"),
    )
)

system = render_keyboard_system(
    manager,
    url_map={
        "toggle-btn": "/toggle",
        "delete-btn": "/delete"
    },
    target_map={
        "toggle-btn": "#list",
        "delete-btn": "#list"
    }
)

assert isinstance(system, KeyboardSystem)
assert system.hints is not None

# Check script
script_html = to_xml(system.script)
assert "handleKeydown" in script_html

# Check hidden inputs
inputs_html = to_xml(system.hidden_inputs)
assert 'id="browser-job-id"' in inputs_html

# Check action buttons
buttons_html = to_xml(system.action_buttons)
assert 'id="toggle-btn"' in buttons_html
assert 'id="delete-btn"' in buttons_html

# Check hints
hints_html = to_xml(system.hints)
assert "Select" in hints_html

In [ ]:
# Test without hints
system_no_hints = render_keyboard_system(
    manager,
    url_map={"toggle-btn": "/toggle", "delete-btn": "/delete"},
    target_map={"toggle-btn": "#list", "delete-btn": "#list"},
    show_hints=False
)

assert system_no_hints.hints is None
assert len(system_no_hints.all_components()) == 3  # No hints

In [ ]:
# Test all_components unpacking
components = system.all_components()
assert len(components) == 4  # script, inputs, buttons, hints

## Quick Setup Function

For simpler use cases with sensible defaults.

In [ ]:
#| export
def quick_keyboard_system(
    zones: tuple[FocusZone, ...],                # focus zones
    actions: tuple[KeyAction, ...],              # keyboard actions
    url_map: dict[str, str],                     # action URLs
    target_map: dict[str, str],                  # action targets
    **kwargs                                     # additional ZoneManager/render options
) -> KeyboardSystem:                             # complete keyboard system
    """Quick setup for simple keyboard navigation."""
    from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
    from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
    
    # Extract manager kwargs
    manager_kwargs = {}
    render_kwargs = {}
    
    manager_keys = {
        'prev_zone_key', 'next_zone_key', 'zone_switch_modifiers',
        'wrap_zones', 'key_mapping', 'initial_zone_id', 'modes',
        'default_mode', 'on_zone_change', 'on_mode_change',
        'on_state_change', 'skip_when_input_focused', 'input_selector',
        'htmx_settle_event', 'expose_state_globally', 'global_state_name',
        'state_hidden_inputs'
    }
    
    for key, value in kwargs.items():
        if key in manager_keys:
            manager_kwargs[key] = value
        else:
            render_kwargs[key] = value
    
    manager = ZoneManager(
        zones=zones,
        actions=actions,
        **manager_kwargs
    )
    
    return render_keyboard_system(
        manager,
        url_map=url_map,
        target_map=target_map,
        **render_kwargs
    )

In [ ]:
# Test quick setup
system = quick_keyboard_system(
    zones=(browser,),
    actions=(
        KeyAction(key=" ", htmx_trigger="select-btn", description="Select"),
    ),
    url_map={"select-btn": "/select"},
    target_map={"select-btn": "#list"},
    show_hints=False
)

assert isinstance(system, KeyboardSystem)
assert system.hints is None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()